In [1]:
import os
os.chdir("..")

In [2]:
%load_ext kedro.extras.extensions.ipython

2022-12-09 19:01:44,419 - root - INFO - Updated path to Kedro project: C:\Users\Lianz\Documents\medgc-tesis
2022-12-09 19:01:45,047 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.


d:\Programs\Anaconda3\envs\medgc-tesis-v2\lib\site-packages\tllib-0.4-py3.9.egg\tllib\alignment\cdan.py:134: DeprecationWarning: invalid escape sequence \o
d:\Programs\Anaconda3\envs\medgc-tesis-v2\lib\site-packages\tllib-0.4-py3.9.egg\tllib\alignment\cdan.py:134: DeprecationWarning: invalid escape sequence \o
d:\Programs\Anaconda3\envs\medgc-tesis-v2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Programs\Anaconda3\envs\medgc-tesis-v2\lib\site-packages\tllib-0.4-py3.9.egg\tllib\modules\grl.py:34: DeprecationWarning: invalid escape sequence \m
d:\Programs\Anaconda3\envs\medgc-tesis-v2\lib\site-packages\tllib-0.4-py3.9.egg\tllib\modules\grl.py:34: DeprecationWarning: invalid escape sequence \m
d:\Programs\Anaconda3\envs\medgc-tesis-v2\lib\site-packages\tllib-0.4-py3.9.egg\tllib\alignment\mdd.py:266: Deprecation

2022-12-09 19:01:52,743 - root - INFO - ** Kedro project MEDGC Tesis
2022-12-09 19:01:52,746 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-12-09 19:01:56,003 - root - INFO - Registered line magic `run_viz`


In [3]:
modelo_resnet_dann = catalog.load("modelo_resnet_dann")

2022-12-09 19:01:56,053 - kedro.io.data_catalog - INFO - Loading data from `modelo_resnet_dann` (PytorchModelDataSet)...


In [4]:
from medgc_tesis.pipelines.modeling.backbone import LeNetBackbone, ResNet18Backbone
from medgc_tesis.pipelines.modeling.models.data import DomainAdaptationDataModule
from torch import nn
from tllib.alignment.dann import ImageClassifier
from tqdm import tqdm
import torch

d:\Programs\Anaconda3\envs\medgc-tesis-v2\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


2022-12-09 19:02:00,977 - pytorch_lightning.utilities.seed - INFO - Global seed set to 48721


In [5]:
backbone = ResNet18Backbone()

In [6]:
dm = DomainAdaptationDataModule(transform=backbone.data_transform())
dm.setup(None)

In [7]:
if hasattr(modelo_resnet_dann, "pool_layer"):
    feature_extractor = nn.Sequential(modelo_resnet_dann.backbone, modelo_resnet_dann.pool_layer, modelo_resnet_dann.bottleneck)
else:
    feature_extractor = nn.Sequential(modelo_resnet_dann.backbone, modelo_resnet_dann.bottleneck)

feature_extractor = feature_extractor.to("cuda")

In [8]:
f_ss = []
y_ss = []
f_tt = []

for idx, batch in enumerate(tqdm(dm.test_dataloader())):
    if idx > 5:
        break

    x_s, y_s = batch["mnist"]
    x_t, y_t = batch["mnist"]

    x_s = x_s.to("cuda")
    x_t = x_t.to("cuda")

    f_s = feature_extractor(x_s)
    f_t = feature_extractor(x_t)

    f_ss.append(f_s)
    y_ss.append(y_s)
    f_tt.append(f_t)

 67%|██████▋   | 6/9 [00:24<00:12,  4.03s/it]


In [12]:
s_features = torch.cat(f_ss, dim=0).cpu().detach().numpy()
s_labels = torch.cat(y_ss, dim=0).cpu().detach().numpy()
t_features = torch.cat(f_tt, dim=0).cpu().detach().numpy()

In [13]:
import umap

mapper = umap.UMAP(random_state=33)
source_feature = mapper.fit_transform(s_features)
target_feature = mapper.transform(t_features)

In [ ]:
import pandas as pd

In [ ]:
mnist = fetch_mldata("MNIST original")
X = mnist.data / 255.0
y = mnist.target
feat_cols = [ 'pixel' + str(i) for i in range(X.shape[1]) ]
df = pd.DataFrame(X,columns=feat_cols)
df['y'] = y
df['label'] = df['y'].apply(lambda i: str(i)) 
X, y = None, None
np.random.seed(42) 
rndperm = np.random.permutation(df.shape[0])

In [ ]:
df_mnist = pd.DataFrame({"0": source_feature[:, 0], "1": source_feature[:, 1]})
df_mnist["label"] = "MNIST"
df_tds = pd.DataFrame({"0": target_feature[:, 0], "1": target_feature[:, 1]})
df_tds["label"] = "TDS"
df_features = pd.concat([df_mnist, df_tds], ignore_index=True)

In [10]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

pixel_cols = df_subset.columns.str.startswith('pixel')
img_w, img_h = 28,28
zoom = 0.5

fig, ax = plt.subplots(figsize=(16,10))
for i,row in df_subset.iterrows():
    image = row[pixel_cols].values.astype(float).reshape((img_w, img_h))
    im = OffsetImage(image, zoom=zoom)
    ab = AnnotationBbox(im, (row["tsne-2d-one"], row["tsne-2d-two"]), xycoords='data', frameon=False)
    ax.add_artist(ab)
    ax.update_datalim([(row["tsne-2d-one"], row["tsne-2d-two"])])
    ax.autoscale()

tensor([4, 7, 4,  ..., 0, 4, 4])